In [1]:
import pandas as pd
import geopandas as gpd 


In [2]:
df=pd.read_csv('./bus.csv',encoding='utf-8') # 버스정류장정보 read

In [3]:
df

,b_nm,b_long,b_lat
0,영주삼거리,129.033262,35.115323
1,영주삼거리,129.033030,35.115283
2,시민아파트,129.031678,35.115075
3,시민아파트,129.032160,35.114921
4,중앙공원.민주공원입구,129.029858,35.114473
...,...,...,...
8517,좌2동행정복지센터,129.183119,35.169047
8518,좌동재래시장,129.172913,35.173195
8519,좌동재래시장,129.173247,35.173347
8520,한마음스포츠센터,129.177295,35.180100


In [4]:
df2=pd.read_csv('./travel_list2.csv',encoding='utf-8') # 관광지정보read
df2

,travel_spot_name,ts_long,ts_lat
0,서면1번가,129.058286,35.153576
1,재난안전체험관,129.105109,35.291822
2,광안리해수욕장,129.118492,35.153791
3,송정해수욕장,129.199640,35.178712
4,해운대해수욕장,129.160308,35.159124
...,...,...,...
559,연제고분판타지축제,129.095056,35.192581
560,다대포어항문화축제,128.972821,35.055623
561,보수동책방골목문화축제,129.027461,35.103125
562,부산고등어축제,129.016761,35.075455


In [5]:
busan_bus_gdf = gpd.GeoDataFrame(df,geometry=gpd.points_from_xy(df.b_long,df.b_lat),crs='epsg:4326')  # WGS84(EPSG:4326) :경도위도좌표계 
busan_travel_gdf = gpd.GeoDataFrame(df2,geometry=gpd.points_from_xy(df2.ts_long,df2.ts_lat),crs='epsg:4326')  

# 부산버스지오df 

In [6]:
busan_travel_gdf

,travel_spot_name,ts_long,ts_lat,geometry
0,서면1번가,129.058286,35.153576,POINT (129.05829 35.15358)
1,재난안전체험관,129.105109,35.291822,POINT (129.10511 35.29182)
2,광안리해수욕장,129.118492,35.153791,POINT (129.11849 35.15379)
3,송정해수욕장,129.199640,35.178712,POINT (129.19964 35.17871)
4,해운대해수욕장,129.160308,35.159124,POINT (129.16031 35.15912)
...,...,...,...,...
559,연제고분판타지축제,129.095056,35.192581,POINT (129.09506 35.19258)
560,다대포어항문화축제,128.972821,35.055623,POINT (128.97282 35.05562)
561,보수동책방골목문화축제,129.027461,35.103125,POINT (129.02746 35.10313)
562,부산고등어축제,129.016761,35.075455,POINT (129.01676 35.07545)


In [7]:
bus_distances = busan_bus_gdf.to_crs(epsg=5186).geometry.distance(busan_travel_gdf.to_crs(epsg=5186).iloc[0].geometry)

# 계산해보기.(서면1번가에서 각 버스정류장 거리계산)
# epsg=5186 : TM좌표계로변환 , m단위의 값을 얻기 위해 4186은 경'도'위'도' 도단위임.
# TM좌표계(투영좌표계) : 3차원 위경도를 2차원평면에 나타내기위해 사용. 우리나라 표준


In [8]:
bus_distances

0        4819.941260
1        4833.935126
2        4913.888201
3        4907.260700
4        5055.119380
            ...     
8517    11507.379600
8518    10672.546524
8519    10705.719319
8520    11239.421322
8521    10792.191496
Length: 8522, dtype: float64

In [9]:
near_bus = df.iloc[bus_distances.idxmin()]['b_nm']
print(f'{busan_travel_gdf.iloc[0].travel_spot_name}에서 가장가까운 버스정류장: {near_bus} ({bus_distances.min()}m)')
# 서면1번가에서 가장가까운 버스정류장: 서면1번가 (37.32908144279374m)


서면1번가에서 가장가까운 버스정류장: 서면1번가 (37.32908144279374m)


In [10]:
# 관광지-가까운 버스정류장 거리계산 함수
def cloest_bus(point_geometry):
    distances =  busan_bus_gdf.to_crs(epsg=5186).geometry.distance(point_geometry)
    return df.iloc[distances.idxmin()]['b_nm'] # idxmin : 최솟값 인덱스찾기

busan_travel_gdf['travel_bus'] = busan_travel_gdf.to_crs(epsg=5186).apply(lambda x: cloest_bus(x.geometry), axis=1)

KeyboardInterrupt: 

In [11]:
busan_travel_gdf

,travel_spot_name,ts_long,ts_lat,geometry,travel_bus
0,서면1번가,129.058286,35.153576,POINT (129.05829 35.15358),서면1번가
1,재난안전체험관,129.105109,35.291822,POINT (129.10511 35.29182),스포원파크
2,광안리해수욕장,129.118492,35.153791,POINT (129.11849 35.15379),광안리해수욕장
3,송정해수욕장,129.199640,35.178712,POINT (129.19964 35.17871),광어골입구
4,해운대해수욕장,129.160308,35.159124,POINT (129.16031 35.15912),해운대해수욕장
...,...,...,...,...,...
559,연제고분판타지축제,129.095056,35.192581,POINT (129.09506 35.19258),중앙하이츠아파트
560,다대포어항문화축제,128.972821,35.055623,POINT (128.97282 35.05562),다대회센터
561,보수동책방골목문화축제,129.027461,35.103125,POINT (129.02746 35.10313),보수동책방골목
562,부산고등어축제,129.016761,35.075455,POINT (129.01676 35.07545),송도해수욕장


In [12]:
busan_travel_gdf.to_csv('./travel_bus.csv',encoding='utf-8')

In [25]:
import folium
from folium import Marker,Circle

In [22]:
m = folium.Map(location=[35.179665,129.0747635],tiles='openstreetmap',zoom_start=11)

In [24]:
for _, row in df2.iterrows():
    Marker(location = [row['ts_lat'], row['ts_long']],
           popup=row['travel_spot_name'],
           icon=folium.Icon(color='blue',icon='cloud')
          ).add_to(m)
m


In [27]:
m = folium.Map(location=[35.179665,129.0747635],tiles='openstreetmap',zoom_start=11)


for _, row in df2.iterrows():
    Circle(location = [row['ts_lat'], row['ts_long']],
           radius = 200,
           popup=row['travel_spot_name'],
    ).add_to(m)

m
